# Initial Analysis

In [45]:
import pandas as pd

# Load the dataset
file_path = '../data/Cell_Phones_&_Accessories.txt.gz'

try:
    df = pd.read_csv(file_path, compression='gzip', sep='\t')
    # Display the first few rows of the dataframe
    print(df.head())
except FileNotFoundError:
    print("File not found. Please ensure the correct file path.")
except Exception as e:
    print(f"An error occurred: {e}")


                       product/productId: B000JVER7W
0  product/title: Mobile Action MA730 Handset Man...
1                             product/price: unknown
2                      review/userId: A1RXYH9ROBAKEZ
3                        review/profileName: A. Igoe
4                            review/helpfulness: 0/0


## Adjust to Data Format

The data is stored as separate lines, with the format `category_name: contained_value`. Each line represents a part of the review, such as the product title, price, or user information.

To make the data more usable, we need to process the file and extract the key-value pairs, grouping them into a structured format (such as a DataFrame) for easier analysis. This includes:
- Parsing each line into key-value pairs.
- Organizing related data (e.g., product information, user reviews) into individual records.
- Converting the data into a format that can be easily analyzed with Python tools like pandas.

The following code performs these steps by reading the file, extracting relevant information, and converting it into a structured DataFrame.


In [46]:
import pandas as pd
import gzip

# Load the dataset
file_path = '../data/Cell_Phones_&_Accessories.txt.gz'

try:
    # Read the .gz file
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        lines = f.readlines()

    # Split each line by ': ' and convert to a dictionary of product attributes
    data = []
    current_review = {}

    for line in lines:
        line = line.strip()
        
        # Check if the line contains two parts (key and value)
        if ": " in line:
            key, value = line.split(": ", 1)
            if key.startswith("product"):
                if current_review:
                    data.append(current_review)  # Append the previous review if it's complete
                current_review = {key: value}  # Start new review
            else:
                current_review[key] = value  # Add additional data to the current review
    
    if current_review:
        data.append(current_review)  # Append the last review if any

    # Convert to DataFrame
    df = pd.DataFrame(data)
    # Display the first few rows
    print(df.head())

except FileNotFoundError:
    print("File not found. Please ensure the correct file path.")
except Exception as e:
    print(f"An error occurred: {e}")


  product/productId                                      product/title  \
0        B000JVER7W                                                NaN   
1               NaN  Mobile Action MA730 Handset Manager - Bluetoot...   
2               NaN                                                NaN   
3        B000JVER7W                                                NaN   
4               NaN  Mobile Action MA730 Handset Manager - Bluetoot...   

  product/price   review/userId review/profileName review/helpfulness  \
0           NaN             NaN                NaN                NaN   
1           NaN             NaN                NaN                NaN   
2       unknown  A1RXYH9ROBAKEZ            A. Igoe                0/0   
3           NaN             NaN                NaN                NaN   
4           NaN             NaN                NaN                NaN   

  review/score review/time review/summary  \
0          NaN         NaN            NaN   
1          NaN         NaN

In [47]:
# Statistics for missing values and 'unknown'
nan_counts = df.isna().sum()  # Count NaN in each column
unknown_counts = (df == 'unknown').sum()  # Count occurrences of 'unknown' in each column

# General information about the data
total_rows = len(df)
total_columns = len(df.columns)

# Display results
print("Number of rows in the dataset:", total_rows)
print("Number of columns in the dataset:", total_columns)

print("\nStatistics for NaN values:")
print(nan_counts)

print("\nStatistics for 'unknown' values:")
print(unknown_counts)

# Percentage of missing data for each column
nan_percentage = (nan_counts / total_rows) * 100
print("\nPercentage of missing data for each column:")
print(nan_percentage)


Number of rows in the dataset: 236243
Number of columns in the dataset: 10

Statistics for NaN values:
product/productId     157313
product/title         157860
product/price         157313
review/userId         157313
review/profileName    157313
review/helpfulness    157313
review/score          157313
review/time           157313
review/summary        157315
review/text           157313
dtype: int64

Statistics for 'unknown' values:
product/productId         0
product/title             0
product/price         43287
review/userId          2276
review/profileName     2276
review/helpfulness        0
review/score              0
review/time               0
review/summary            0
review/text               0
dtype: int64

Percentage of missing data for each column:
product/productId     66.589486
product/title         66.821028
product/price         66.589486
review/userId         66.589486
review/profileName    66.589486
review/helpfulness    66.589486
review/score          66.58948

In [48]:
print("Examples where 'review/text' is NaN:")
print(df[df['review/text'].isna()].head(5))

print("\nExamples where 'product/price' is 'unknown':")
print(df[df['product/price'] == 'unknown'].head(5))

print("\nExamples with multiple NaNs in a row:")
print(df[df.isna().sum(axis=1) > 7].head(5))


Examples where 'review/text' is NaN:
  product/productId                                      product/title  \
0        B000JVER7W                                                NaN   
1               NaN  Mobile Action MA730 Handset Manager - Bluetoot...   
3        B000JVER7W                                                NaN   
4               NaN  Mobile Action MA730 Handset Manager - Bluetoot...   
6        B000JVER7W                                                NaN   

  product/price review/userId review/profileName review/helpfulness  \
0           NaN           NaN                NaN                NaN   
1           NaN           NaN                NaN                NaN   
3           NaN           NaN                NaN                NaN   
4           NaN           NaN                NaN                NaN   
6           NaN           NaN                NaN                NaN   

  review/score review/time review/summary review/text  
0          NaN         NaN         

# Framed Data Summary

In [49]:
# Load the dataset
file_path = '../data/cleaned_amazon_reviews.csv'

try:
    df = pd.read_csv(file_path)
    # Display the first few rows of the dataframe
    print(df.head())
except FileNotFoundError:
    print("File not found. Please ensure the correct file path.")
except Exception as e:
    print(f"An error occurred: {e}")

  product/productId                                      product/title  \
0        B000JVER7W  Mobile Action MA730 Handset Manager - Bluetoot...   
1        B000JVER7W  Mobile Action MA730 Handset Manager - Bluetoot...   
2        B000JVER7W  Mobile Action MA730 Handset Manager - Bluetoot...   
3        B000924R5I  USB Data Cable for Sony-Ericsson Z600, Z500, Z...   
4        B000924R5I  USB Data Cable for Sony-Ericsson Z600, Z500, Z...   

   product/price   review/userId                  review/profileName  \
0            NaN  A1RXYH9ROBAKEZ                             A. Igoe   
1            NaN   A7L6E1KSJTAJ6                        Steven Martz   
2            NaN  A2NOQQ7I6PU7CW  Daniel M. Johnson "rocknbluesharp"   
3            NaN  A29Q0CLOF0U8BN                            E. Owens   
4            NaN   ANNHJND2MHUI3         Isaac Salas "=CRBF=gB^link"   

  review/helpfulness  review/score review/time  \
0                0/0           1.0  2009-01-31   
1                0/0  

## We Need to Take Care of Problematic Data
- This segment requires 'src/preprocessing.py' to run

In [50]:

print("\nNull values in the dataset:")
print(df.isnull().sum())



Null values in the dataset:
product/productId                     0
product/title                       547
product/price                     43287
review/userId                         0
review/profileName                    3
review/helpfulness                    0
review/score                          0
review/time                           0
review/summary                        3
review/text                           0
review/helpfulness_numerator          0
review/helpfulness_denominator        0
dtype: int64


In [51]:
# Number of unique values in each column
for col in df.columns:
    print(f"Number of unique values in {col}: {df[col].nunique()}")


Number of unique values in product/productId: 7438
Number of unique values in product/title: 7149
Number of unique values in product/price: 943
Number of unique values in review/userId: 68041
Number of unique values in review/profileName: 63367
Number of unique values in review/helpfulness: 1305
Number of unique values in review/score: 5
Number of unique values in review/time: 4431
Number of unique values in review/summary: 60319
Number of unique values in review/text: 77390
Number of unique values in review/helpfulness_numerator: 197
Number of unique values in review/helpfulness_denominator: 209


## UserID and profile name being unknown should be treated as NA

In [52]:
df['review/userId'] = df['review/userId'].replace('unknown', pd.NA)
df['review/profileName'] = df['review/profileName'].replace('unknown', pd.NA)


In [53]:
# Description of the dataset 
print("\nDescription of the dataset:")
print(df.describe(include='all'))  # include='all' includes text columns as well


Description of the dataset:
       product/productId                                      product/title  \
count              78930                                              78383   
unique              7438                                               7149   
top           B0009B0IX4  Plantronics Voyager 510 Bluetooth Headset [Ret...   
freq                3443                                               3443   
mean                 NaN                                                NaN   
std                  NaN                                                NaN   
min                  NaN                                                NaN   
25%                  NaN                                                NaN   
50%                  NaN                                                NaN   
75%                  NaN                                                NaN   
max                  NaN                                                NaN   

        product/price 

In [54]:
# Find the rows with the lowest prices and the rows with missing prices
lowest_prices = df[df['product/price'] == df['product/price'].min()]
missing_prices = df[df['product/price'].isnull()]

# Display the results
lowest_prices.head(10), missing_prices.head(10)


(     product/productId                                      product/title  \
 3004        B000E7FLYY  USB Data Cable for Sony-Ericsson W800/ W800C/ ...   
 3599        B00030KI9U  Blackberry 8310 8320 8330 Curve 8100 8110 8120...   
 3600        B00030KI9U  Blackberry 8310 8320 8330 Curve 8100 8110 8120...   
 3601        B00030KI9U  Blackberry 8310 8320 8330 Curve 8100 8110 8120...   
 3602        B00030KI9U  Blackberry 8310 8320 8330 Curve 8100 8110 8120...   
 3603        B00030KI9U  Blackberry 8310 8320 8330 Curve 8100 8110 8120...   
 3633        B000Q8XSVO           Inspire's Car Charger for Microsoft Zune   
 3634        B000Q8XSVO           Inspire's Car Charger for Microsoft Zune   
 3635        B000Q8XSVO           Inspire's Car Charger for Microsoft Zune   
 3636        B000Q8XSVO           Inspire's Car Charger for Microsoft Zune   
 
       product/price   review/userId                review/profileName  \
 3004           0.01   AGTJNSIHGD6FT                    Miles Eric

## There seems to be a problem with the dataset, as the product prices are irrelevant to actual prices.
- There is a necessity to take some actions because they are highly imprecise.

In [55]:
# Check for products with the same productId but different prices
product_id_groups = df.groupby('product/productId')['product/price'].nunique()
products_diff_price_id = product_id_groups[product_id_groups > 1].index
df_diff_price_id = df[df['product/productId'].isin(products_diff_price_id)]

# Check for products with the same title but different prices
product_title_groups = df.groupby('product/title')['product/price'].nunique()
products_diff_price_title = product_title_groups[product_title_groups > 1].index
df_diff_price_title = df[df['product/title'].isin(products_diff_price_title)]

# Display results
print("Products with same productId but different prices:")
print(df_diff_price_id[['product/productId', 'product/title', 'product/price']].drop_duplicates())

print("\nProducts with same title but different prices:")
print(df_diff_price_title[['product/title', 'product/price']].drop_duplicates())


Products with same productId but different prices:
Empty DataFrame
Columns: [product/productId, product/title, product/price]
Index: []

Products with same title but different prices:
                                           product/title  product/price
19     Samsung T809 M620 i607 U420 A303 A717 Car Char...           6.75
766    LG OEM LGIP-A1000E BATTERY FOR VX4700 VX4750 V...           4.00
895    Motorola OEM AANN4285B BATTERY FOR V150 C650 V180           0.97
985      Motorola OEM SNN5705 BATTERY FOR i930 i870 i530           5.45
1002     Motorola OEM SNN5683A BATTERY FOR V60 V500 V551           6.99
...                                                  ...            ...
72241  Battery-Biz Inc. 3.7 Volt Li-Ion Cellular Phon...           6.76
73856  NEW OEM SAMSUNG TAD037JBE HOME CHARGER SGH-X47...           8.71
76513  USB Data Cable for Nokia 3230/ 3300/ 6170/ 623...           4.75
76663                     Nokia CA-53 Connectivity Cable           4.47
77365             Motoro

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# import pandas as pd

# # Load the dataset
# df = pd.read_csv(cleaned_file_path)

# # 1. Remove rows with missing titles
# df = df.dropna(subset=['product/title'])

# # 2. Preprocess titles
# vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
# tfidf_matrix = vectorizer.fit_transform(df['product/title'])

# # 3. Calculate similarities between products - sparse matrix
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# # 4. Adjust price for products without a price
# def adjust_price_for_missing(df, cosine_sim):
#     for i in range(len(df)):
#         if pd.isna(df['product/price'][i]):
#             # Find similar products
#             similar_indices = np.argsort(cosine_sim[i])[::-1]  # Sort by similarity
#             similar_prices = [df['product/price'][idx] for idx in similar_indices if not pd.isna(df['product/price'][idx])]
            
#             if similar_prices:
#                 # Calculate the average price of similar products
#                 adjusted_price = np.mean(similar_prices)
#                 df.at[i, 'product/price'] = adjusted_price
#     return df

# # 5. Apply the price adjustment to missing products
# df = adjust_price_for_missing(df, cosine_sim)
# print(df)


## Issues with Product Prices

We don't have any product IDs to copy prices from. Using similar titles might be inconsistent and needs reconsideration.

- Web scraping to find actual prices is time-consuming and not necessarily effective.
- We will drop the price column.
- The above code is too memory-intensive to run, meaning we don't have anything left.

In [ ]:
# Save the cleaned dataset to a new CSV file
df = df.drop(columns=['product/price'])  # Drop the 'product/price' column
cleaned_file_path = '../data/cleaned_amazon_reviews_v2.csv'

df.to_csv(cleaned_file_path, index=False)
cleaned_file_path
